In [131]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
# from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
# import numpy as np
# import sklearn
import pickle
from keras.applications import *
from keras.preprocessing.image import *
# importing dependencies
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import tqdm ##
from tqdm import * ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle

In [137]:
folder = 'data/ens_unverified/' #共同predict對unverified data的結果
type_ = 'mfcc' #要抽取哪一個種類的unverified trainX出來去re-train

acc_df = pd.read_csv('data/ens_unverified/validation_ACC_P1S1.csv')
acc_df.columns = ['model','csv_name','acc','other']
acc_df = acc_df.filter(['csv_name','acc'])
files = os.listdir(folder)

ratio_all=0
for i,csv in enumerate(files):
    if csv.startswith('validation_ACC'):
        continue
    else:
        ratio = acc_df[acc_df['csv_name'] == csv]['acc'].values[0]
#         print(ratio)
        ratio_all += ratio
    df = pd.read_csv(os.path.join(folder,csv),header=None)
    if df.iloc[0,0] == 'fname':
        df = df.drop(0,axis=0)
    df = df.drop(0,axis=1)

    
    if i==0:
        train_X = df.values*ratio
    else:
        train_X += df.values*ratio 
print(train_X.shape)
train_X = train_X / ratio_all

(5763, 41)


In [138]:
df = pd.read_csv('data/train_label.csv')
df = df[df['manually_verified']==0]
df.reset_index(drop=True,inplace=True)
df['label_verified'] = "none"
df['verified_confidence']=0.0
# df_ = pd.DataFrame(columns=['label_verified','verified_confidence'])
for i,r in df.iterrows():
    df.iloc[i,3] = np.argmax(train_X[i])
    df.iloc[i,4] = max(train_X[i])
#     print(i)
#     print(r['label'])
df

,fname,label,manually_verified,label_verified,verified_confidence
0,00044347.wav,Hi-hat,0,0,0.928646
1,002d256b.wav,Trumpet,0,40,0.489762
2,003b91e8.wav,Cello,0,4,0.886710
3,004ad66f.wav,Clarinet,0,7,0.437608
4,0063ab88.wav,Computer_keyboard,0,24,0.526737
5,0075d39c.wav,Keys_jangling,0,9,0.740670
6,00780200.wav,Snare_drum,0,10,0.834828
7,0079d310.wav,Writing,0,24,0.379398
8,00ad7068.wav,Laughter,0,12,0.887216
9,00c5808a.wav,Tearing,0,24,0.263318


In [139]:
dicts_ch = pickle.load(open('data/map_reverse.pkl','rb'))
df['label_verified'] = df['label_verified'].map(dicts_ch)
threshold_upper = df['verified_confidence'].mean() + df['verified_confidence'].std()
threshold_lower = df['verified_confidence'].mean()
df['take']=0
for i,r in df.iterrows():
    if r['verified_confidence']>threshold_upper:
        df.iloc[i,5]=1
    elif ((r['label'] == r['label_verified'])and(r['verified_confidence']>threshold_lower)):
        df.iloc[i,5]=1
    else:
        df.iloc[i,5]=0
df
# if df['label']==df['verified_confidence']:
#     df['keep']=1

,fname,label,manually_verified,label_verified,verified_confidence,take
0,00044347.wav,Hi-hat,0,Hi-hat,0.928646,1
1,002d256b.wav,Trumpet,0,Finger_snapping,0.489762,0
2,003b91e8.wav,Cello,0,Cello,0.886710,1
3,004ad66f.wav,Clarinet,0,Clarinet,0.437608,0
4,0063ab88.wav,Computer_keyboard,0,Scissors,0.526737,0
5,0075d39c.wav,Keys_jangling,0,Keys_jangling,0.740670,1
6,00780200.wav,Snare_drum,0,Snare_drum,0.834828,1
7,0079d310.wav,Writing,0,Scissors,0.379398,0
8,00ad7068.wav,Laughter,0,Laughter,0.887216,1
9,00c5808a.wav,Tearing,0,Scissors,0.263318,0


In [140]:
df_ = df[df['take']==1]
df_ = df_.filter(['fname','label_verified'])
df_.to_csv('feature/'+type_+'/Y_train_ens_verified.csv') # 儲存train_Y
train_X = np.load('feature/'+type_+'/X_train.npy')
df_label = pd.read_csv('data/train_label.csv')
df = pd.merge(df_label,df_,on='fname',how='left')
df_

,fname,label,manually_verified,label_verified
0,00044347.wav,Hi-hat,0,Hi-hat
1,001ca53d.wav,Saxophone,1,NaN
2,002d256b.wav,Trumpet,0,NaN
3,0033e230.wav,Glockenspiel,1,NaN
4,00353774.wav,Cello,1,NaN
5,003b91e8.wav,Cello,0,Cello
6,003da8e5.wav,Knock,1,NaN
7,0048fd00.wav,Gunshot_or_gunfire,1,NaN
8,004ad66f.wav,Clarinet,0,NaN
9,0063ab88.wav,Computer_keyboard,0,NaN


In [141]:
df = df[df['label_verified'].notnull()]
train_X = train_X[df.index]
print(train_X.shape)
np.save('feature/'+type_+'/X_train_ens_verified.npy',train_X)  ## 儲存train_X
df

(2201, 40, 345, 1)


,fname,label,manually_verified,label_verified
0,00044347.wav,Hi-hat,0,Hi-hat
5,003b91e8.wav,Cello,0,Cello
11,0075d39c.wav,Keys_jangling,0,Keys_jangling
12,00780200.wav,Snare_drum,0,Snare_drum
16,00ad7068.wav,Laughter,0,Laughter
20,00c9e799.wav,Oboe,0,Oboe
22,00ce569f.wav,Hi-hat,0,Hi-hat
26,00d9fa61.wav,Flute,0,Flute
33,0120d246.wav,Bus,0,Bus
38,013c3135.wav,Clarinet,0,Clarinet


,fname,label,manually_verified,label_verified
0,00044347.wav,Hi-hat,0,Hi-hat
5,003b91e8.wav,Cello,0,Cello
11,0075d39c.wav,Keys_jangling,0,Keys_jangling
12,00780200.wav,Snare_drum,0,Snare_drum
16,00ad7068.wav,Laughter,0,Laughter
20,00c9e799.wav,Oboe,0,Oboe
22,00ce569f.wav,Hi-hat,0,Hi-hat
26,00d9fa61.wav,Flute,0,Flute
33,0120d246.wav,Bus,0,Bus
38,013c3135.wav,Clarinet,0,Clarinet


(2205, 40, 345, 1)

***

In [59]:
np.argmax(train_X[0]) , max(train_X[0])

(0, 0.9387134916386914)

In [88]:
df['verified_confidence'].mean()

0.5583730920813336

In [94]:
df['take']=0
df_ = df[df['label']==df['label_verified']]
df_['take']=1
# .loc['take']=1
#     print('kk')
df_

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,fname,label,manually_verified,label_verified,verified_confidence,take
0,00044347.wav,Hi-hat,0,Hi-hat,0.938713,1
2,003b91e8.wav,Cello,0,Cello,0.896771,1
3,004ad66f.wav,Clarinet,0,Clarinet,0.447639,1
5,0075d39c.wav,Keys_jangling,0,Keys_jangling,0.750052,1
6,00780200.wav,Snare_drum,0,Snare_drum,0.835238,1
8,00ad7068.wav,Laughter,0,Laughter,0.897087,1
10,00c82919.wav,Fart,0,Fart,0.296973,1
11,00c9e799.wav,Oboe,0,Oboe,1.006394,1
12,00ce569f.wav,Hi-hat,0,Hi-hat,0.902627,1
13,00d1fe46.wav,Cough,0,Cough,0.516341,1
